<a href="https://colab.research.google.com/github/jscd98/explorer/blob/main/stocks_vs_bonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################
### stocks vs bonds strategy ###
################################

'''
Strategy calculates the momentum for all three assets; stocks, bonds & cash.
It allocates a larger allocation to the highest positive momentum score & a smaller
allocation to the 2nd best positive momentum score. Any unallocated amounts are put into cash.
The user simply needs to pick the assest for their exposure to Stocks, Bonds & Cash & also the ratio
of large vs small allocation. Default is SPY, TLT & BIL and 80% vs 20%.
'''

# Select tickers for 'Stocks', 'Bond' & 'Cash'
stocks = 'SPY'
bonds = 'IEF'
cash = 'BIL'

# Select ratio for large & small allocation.
large_allocation = 0.80
small_allocation = 0.20

# Filter etf price history for the tickers selected
svb_ticker_list = [stocks, bonds, cash]
svb_rtn = etf_pct_returns[svb_ticker_list]

# Calculate the momentum score defined by 'calculate_mom13612E' above
svb_mom_score = momentum_score[svb_ticker_list]

# Create a masking scheme to apply the rules defined above, this gives a dataframe of weights 'svb_selection'
  # Create df populated with 0's, that is the same size as the 'svb_mom_score'
svb_selection = pd.DataFrame(0, index=svb_mom_score.index,
                             columns=svb_mom_score.columns)
  # Mask Stocks' column depending on it's momentum score
svb_stock_mask = ((svb_mom_score[stocks] > svb_mom_score[bonds]) & (svb_mom_score[stocks] > 0)) * large_allocation + (
            (svb_mom_score[stocks] < svb_mom_score[bonds]) & (svb_mom_score[stocks] > 0)) * small_allocation
svb_selection[stocks] = svb_stock_mask
  # Mask 'bond' column depending on it's momentum score
svb_bond_mask = ((svb_mom_score[bonds] > svb_mom_score[stocks]) & (svb_mom_score[bonds] > 0)) * large_allocation + (
        (svb_mom_score[bonds] < svb_mom_score[stocks]) & (svb_mom_score[bonds] > 0)) * small_allocation
svb_selection[bonds] = svb_bond_mask
  # Mask the Cash column with 1 - the sum of the values allocated to stocks & bonds (if any)
svb_selection[cash] = 1 - svb_selection[[stocks, bonds]].sum(axis=1)

# Returns a dataframe of asset allocation weights as specified by the calculations above
svb_weights = svb_selection[svb_selection.index >= start_date_plus_12m]
svb_returns = (svb_weights.shift(1) * svb_rtn[svb_rtn.index >= start_date_plus_12m]).sum(axis=1)
svb_cum_sum  = (1 + svb_returns).cumprod()
# qs.reports.full(svb_returns)
# print(svb_cum_sum.to_string())

from matplotlib import pyplot as plt
svb_cum_sum.plot(kind='line', figsize=(8, 4), title='SVB P&L')
plt.yscale('log')
plt.gca().spines[['top', 'right']].set_visible(False)